In [13]:
import numpy as np

"""
Equation Set 1: Fire Simulation using Huygens' Principle Geometry
                These equations are primarily related to eliptical geometry, and used to physically model
                the fire and allow it to be drawn as a polygon on the surface
                
Equation Set 2: Vectoring wind and Slope
                These are straightforward geometric calculations applied to wind and slope to rectify
                coordinate differences between the horizontal plane of Huygens Geometry and the real life plane
                
Equation Set 3: Eliptical Geometry
                These equations compute a,b,c and the elipse parameters used in equation set 1
                
# equation sets 4 and onward pertain to empirical fire models, not abstract geometry

Equation Set 4: Surface Fire - Rothermel (1972)
                These equations model surface fire spread based on wind, slope, and fuel type.
                Spread is computed on a plane parallel to the ground at every vertex of our huygens polygon.
                
Equation Set 5: Crown Fire - Van Wagner (1977, 1993)
                These equations determine whether fire makes a transition to burning in crown fuels,
                and whether it spreads actively or through tree crowns or simply scorches individual trees

Equation Set 6: Fire Acceleration

Equation Set 7: Spotting

Equation Set 8: Misc Weather Stuff

Variables that are unaccounted for:
    - fill in
"""

# Equation Set 1:
# Fire Simulation using Huygens' Principle Geometry

def eq_1(xs, ys, theta, a, b, c):
    """
    :param xs: x coordinate of orientation of vertex on fire front (m)
    :param ys: y coordinate of orientation of vertex on fire front (m)
    :param theta: direction of maximum fire spread rate in polar coordinates (radians azimuthal)
    :param a: defines shape of eliptical fire, from eq_15 (vertical radius) (m/min)
    :param b: defines shape of eliptical fire, from eq_16 (horizontal radius) (m/min)
    :param c: defines shape of eliptical fire, from eq_17 (distance to ignition point) (m/min)
    :return: Xt, orthogonal spread rate differential (m/min)
    """
    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)
    numerator = a*a*cos_theta*(xs*sin_theta + ys*cos_theta) - b*b*sin_theta*(xs*cos_theta - ys*sin_theta)
    denominator = (b*b*(xs*cos_theta + ys*sin_theta)**2 - a*a*(xs*sin_theta - ys*cos_theta)**2)**.5
    return numerator/denominator + c*sin_theta

def eq_2(xs, ys, theta, a, b, c):
    """
    :param xs: x coordinate of orientation of vertex on fire front (m)
    :param ys: y coordinate of orientation of vertex on fire front (m)
    :param theta: direction of maximum fire spread rate in polar coordinates (radians azimuthal)
    :param a: defines shape of eliptical fire, from eq_15 (vertical radius) (m/min)
    :param b: defines shape of eliptical fire, from eq_16 (horizontal radius) (m/min)
    :param c: defines shape of eliptical fire, from eq_17 (distance to ignition point) (m/min)
    :return: Yt, orthogonal spread rate differential (m/min)
    """
    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)
    numerator = a*a*sin_theta*(xs*sin_theta + ys*cos_theta) - b*b*cos_theta*(xs*cos_theta - ys*sin_theta)
    denominator = (b*b*(xs*cos_theta + ys*sin_theta)**2 - a*a*(xs*sin_theta - ys*cos_theta)**2)**.5
    return numerator/denominator + c*cos_theta

def eq_3(x_im1, x_ip1, Di, omega_i):
    """
    :param x_im1: coordinate of i-1 vertex of fire front
    :param x_ip1: coordinate of i+1 vertex of fire front
    :param Di: Slope correction constant for i, from eq_5 (meters)
    :param omega_i: 'aspect' of ith vertex (radians)
    :return: xs, x component of direction normal to fire front for each vertex (xi, yi)
    """
    return (x_im1 - x_ip1) + Di*np.sin(omega_i)

def eq_4(y_im1, y_ip1, Di, omega_i):
    """
    :param y_im1: coordinate of i-1 vertex of fire front
    :param y_ip1: coordinate of i+1 vertex of fire front
    :param Di: Slope correction constant for i, from eq_5 (meters)
    :param omega_i: 'aspect' of ith vertex (radians)
    :return: ys, y component of direction normal to fire front for each vertex (xi, yi)
    """
    return (y_im1 - y_ip1) + Di*np.cos(omega_i)

def eq_5(x_im1, x_ip1, y_im1, y_ip1, delta_i, phi_i):
    """
    :param x_im1: coordinate of i-1 vertex of fire front
    :param x_ip1: coordinate of i+1 vertex of fire front
    :param y_im1: coordinate of i-1 vertex of fire front
    :param y_ip1: coordinate of i+1 vertex of fire front
    :param delta_i: difference between aspect direction (omega_i) and orientation angle (alpha_i), from eq_6 (radians)
    :param phi_i: local slope in aspect (omega_i) direction (radians)
    :return: Di, slope correction constant for i (meters)
    """
    mag_diff = ((x_im1 - x_ip1)**2+(y_im1 - y_ip1)**2)**.5
    return mag_diff*np.cos(delta_i)*(1 - np.cos(phi_i))

def eq_6(omega_i, alpha_i, phi_i):
    """
    :param omega_i: 'aspect' of ith vertex (radians)
    :param alpha_i: orientation angle of perimeter segment, from eq_7 (radians)
    :param phi_i: local slope in aspect (omega_i) direction (radians)
    :return: delta_i, difference between aspect direction (omega_i) and orientation angle (alpha_i) (radians)
    """
    return np.arctan(np.tan(omega_i - alpha_i)/np.cos(phi_i))

def eq_7(x_im1, x_ip1, y_im1, y_ip1):
    """
    :param x_im1: coordinate of i-1 vertex of fire front
    :param x_ip1: coordinate of i+1 vertex of fire front
    :param y_im1: coordinate of i-1 vertex of fire front
    :param y_ip1: coordinate of i+1 vertex of fire front
    :return: alpha_i, the orientation angle
    """
    return np.arctan((y_im1 - y_ip1)/(x_im1 - x_ip1))

def eq_8(Xt, Dr, omega_i):
    """
    :param Xt: orthogonal spread rate differential (m/min)
    :param Dr: difference in spread rates between horizontal plane and local sloping plane (m/min)
    :param omega_i: 'aspect' of ith vertex (radians)
    :return: Xt_prime, spread rate differential on horizontal plane (m/min)
    """
    return Xt + Dr*np.sin(omega_i)

def eq_9(Yt, Dr, omega_i):
    """
    :param Yt: orthogonal spread rate differential (m/min)
    :param Dr: difference in spread rates between horizontal plane and local sloping plane, from eq_10 (m/min)
    :param omega_i: 'aspect' of ith vertex (radians)
    :return: Yt_prime, spread rate differential on horizontal plane (m/min)
    """
    return Yt + Dr*np.cos(omega_i)

def eq_10(Xt, Yt, omega_i, phi_i):
    """
    :param Xt: orthogonal spread rate differential (m/min)
    :param Yt: orthogonal spread rate differential (m/min)
    :param omega_i: 'aspect' of ith vertex (radians)
    :param phi_i: local slope in aspect (omega_i) direction (radians)
    :returns: Dr, difference in spread rates between horizontal plane and local sloping plane (m/min)
    """
    diff = (Xt**2 + Yt**2)**.5
    cos_part = np.cos(omega_i - np.arctan(Yt/Xt))
    return diff*cos_part*(1-np.cos(phi_i))

# Equation Set 2
# Vectoring Wind and Slope

def eq_11(beta, phi):
    """
    :param beta: packing ratio of fuel bed (from fuel type data)
    :param phi: slope (from topographical data) (radians - probably will have to calculate gradient of elev data)
    :return: Phi_s, dimensionless coefficient of slope
    """
    return 5.275*beta**(-.3)*np.tan(phi)**2 # AMBIGUITY WITH SQUARE / TANGENT ORDER, POTENTIAL BUG

def eq_12(C, U, B, beta, beta_op, E):
    """
    :param C: Function of fuel partical size in fuel bed
    :param U: Midflame wind speed (meters per second)
    :param B: Function of fuel partical size in fuel bed
    :param beta: packing ratio of fuel bed (from fuel type data)
    :param beta_op: optimum packing ratio
    :param E: Function of fuel partical size in fuel bed
    :return: Phi_w, dimensionless coefficient for midflame wind speed
    """
    # B = 40 ?
    CUB = C*(3.281*U)**B
    return CUB*(beta/beta_op)**(-E)

# Equation set 3
# Ellipcal Dimensions

def eq_13(U):
    """
    :param U: midlfame wind speed (meters per second)
    :return: LB, length to breadth ratio of flames 
    """
    return .936*np.exp(.2566*U) + .461*np.exp(-.1548*U) - .397

def eq_14(LB):
    """
    :param LB: length to breadth ratio of flames
    :return HB: head to back ratio
    """
    numerator = (LB + (LB**2 - 1)**.5)
    denominator = (LB - (LB**2 - 1)**.5)
    return numerator/denominator

def eq_15(R, HB, LB):
    """
    :param R: Fire spread rate, from eq_18 (meters per minute)
    :param HB: head to back ratio, from eq_14
    :param LB: length to breadth ratio of flames, from eq_13
    :return a: defines shape of eliptical fire (vertical radius) (m/min)
    """
    return .5*(R + R/B)/LB

def eq_16(R, HB):
    """
    :param R: Fire spread rate, from eq_18 (m/min)
    :param HB: head to back ratio, from eq_14
    :return b: defines shape of eliptical fire (horizontal radius) (m/min)
    """
    return (R + R/HB)/2.0

def eq_17(R, HB, b):
    """
    :param R: Fire spread rate, from eq_18 (m/min)
    :param HB: head to back ratio, from eq_14
    :param b: defines shape of eliptical fire, from eq_16 (horizontal radius) (m/min)
    :return c: defines shape of eliptical fire (distance to ignition point) (m/min)
    """
    return b - R/HB

# Equation Set 4
# Fire Behavior Models

def eq_18(IR, xi, rho_b, epsilon, Q_ig, Phi_w, Phi_s, R):
    """
    :param IR: Reacton Intensity (Kj/min/m^2)
    :param xi: propogating flux ratio, unitless
    :param rho_b: ovendry bulk density (kg/m^3)
    :param epsilon: effective heating number, dimensionless
    :param Q_ig: heat of pre-ignition (Kj/Kg)
    :param Phi_w: dimensionless coefficient for midflame wind speed, from eq_11
    :param Phi_s: dimensionless coefficient of slope, from eq_12
    :return: R, fire spread rate (m/min)
    """
    num = IR*xi*(1 + Phi_w + Phi_s)
    den = rho_b*epsilon*Q_ig
    return num/den

def eq_19(h, w, R):
    """
    :param h: heat yield of fuel (Kj/Kg)
    :param w: weight of fuel per unit area (Kg/m^2)
    :param R: fire spread rate, from eq_18 (m/min)
    :return: Ib, fireline intensity (Kw/m)
    """
    return h*w*R/60

def eq_20(IR, R, sigma):
    """
    :param IR: Reacton Intensity (Kj/min/m^2)
    :param R: fire spread rate, from eq_18 (m/min)
    :param sigma: surface area to volume ratio of fuel bed (m^-1)
    :return: Ib, fireline intensity (Kw/m)
    """
    return (IR/60)*(12.6*R/sigma)

# Equation Set 5
# Crown Fire

def eq_21(CBH, M):
    """
    :param M: crown foliar moisture constant (percentage)
    :param CBH: Height to Crown Base (m)
    :return: Io, threshold for transition to crown fire (Kw/m)
    """
    return (.01*CBH*(460 + 25.9*M))**1.5

def eq_22(CBD):
    """
    :param CBD: crown bulk density (Kg/m^3)
    :return: RAC, fire spread rate in crown (m/min)
    """
    return 3.0/CBD

def eq_23(R, CFB, R_Cmax):
    """
    :param R: fire spread rate, from eq_18 (m/min)
    :param CFB: Crown Fraction Burned, from eq_25
    :param R_Cmax: maximum crown fire spread rate, from eq_24 (m/min)
    :return: R_Cactual, actual crown fire spread (m/min)
    """
    return R + CFB*(R_Cmax - R)

def eq_24(R_10, Ei):
    """
    :param R_10: Active crown fire spread rate (m/min)
    :param Ei: Fraction of forward crown fire spread rate at ith index
    :return: R_Cmax, maximum crown fire spread rate (m/min)
    """
    return 3.34*R_10*Ei

def eq_25(a_c, R, Ro):
    """
    :param a_c: scaling coefficient for crown fraction burn, from eq_26
    :param R: fire spread rate, from eq_18 (m/min)
    :param Ro: crital surface fire spread rate, from eq_27 (m/min)
    :return: CFB, Crown Fraction Burned
    """
    return (1 - np.exp(-a_c*(R-Ro)))

def eq_26(RAC, Ro):
    """
    :param RAC: fire spread rate in crown, from eq_22 (m/min)
    :param Ro: crital surface fire spread rate, from eq_27 (m/min)
    :return: a_c, scaling coefficient for crown fraction burn
    """
    return -np.log(.1)/(.9*(RAC-Ro))

def eq_27(R, Ib, Io):
    """
    :param R: fire spread rate, from eq_18 (m/min)
    :param Ib: fireline intensity, from eq_20 (Kw/m)
    :param Io: threshold for transition to crown fire, from eq_21 (Kw/m)
    :return: Ro, crital surface fire spread rate (m/min)
    """
    return Io*(R/Ib)

def eq_28(Ib, R, CFB, CBD, H, CBH, R_Cactual):
    """
    :param Ib: fireline intensity, from eq_20 (Kw/m)
    :param R: fire spread rate, from eq_18 (m/min)
    :param CFB: Crown Fraction Burned, from eq_25
    :param CBD: crown bulk density (Kg/m^3)
    :param H: Crown Height (m)
    :param CBH: Height to Crown Base (m)
    :param R_Cactual: actual crown fire spread, from eq_23 (m/min)
    :return: Ic, intensity of crown fire (Kw/m)
    """
    return 300*(Ib/(300*R) + CFB*CBD*(H-CBH))*R_Cactual

# Equation Set 6
# Fire Acceleration

def eq_29(R, a_a, t):
    """
    :param R: fire spread rate, from eq_18 (m/min)
    :param a_a: Constant that determines rate of acceleration (suggested @ .115 and .3)
    :param t: elapsed time (m)
    :return: Rt, fire spread rate at a given time t (m/min)
    """
    return R*(1 - np.exp(-a_a*t))

def eq_30(a_a, CFB):
    """
    :param a_a: Constant that determines rate of acceleration (suggested @ .115 and .3)
    :param CFB: Crown Fraction Burned, from eq_25
    :return: a_a (wut?)
    """
    return a_a - (18.8*CFB**2.5)*np.exp(-8*CFB)

def eq_31(R, a_a, t, a):
    """
    :param R: fire spread rate, from eq_18 (m/min)
    :param a_a: Constant that determines rate of acceleration (suggested @ .115 and .3)
    :param t: elapsed time (m)
    :param a: defines shape of eliptical fire, from eq_15 (vertical radius) (m/min)
    :return: D, the spread distance (m)
    """
    return R*(t + (np.exp(-a*t)/a_a) - (1/a_a))

def eq_32(a_a, R, Tt, D_tp1):
    """
    :param R: fire spread rate, from eq_18 (m/min)
    :param a_a: Constant that determines rate of acceleration (suggested @ .115 and .3)
    :param Tt: time required to achieve current spread rate, from eq_33 (min)
    :param D_tp1: Spread distance at next timestep (m)
    :return: Dt, spread distance at current timestep (m)
    """
    return R*(Tt + (np.exp(-a*Tt)/a_a) - (1/a_a)) + D_tp1

def eq_33(R, Rt, a_a):
    """
    :param R: fire spread rate, from eq_18 (m/min)
    :param Rt: fire spread rate at a given time t, from eq_29 (m/min)
    :param a_a: Constant that determines rate of acceleration (suggested @ .115 and .3)
    :return: Tt, time required to achieve current spread rate (min)
    """
    return np.log((1-Rt)/R)/a_a

# Equation Set 7
# Spotting

def eq_34(t_o, z, z_F):
    """
    :param t_o: time of steady burning of tree crowns (min)
    :param z: particul height (m)
    :param z_F: flame height (m)
    :return: t_f, duration of bouyant flow structure of torching tree (min)
    """
    return t_o + 1.2 + (5.963/3.)*(((4.563 + z/z_F)/(5.963))**1.5 - 1)

def eq_35(t_o, t1, t2, t3):
    """
    :param t_o: time of steady burning of tree crowns (min)
    :param t1: time for partical to travel from initial height to flame tip, from eq_36 (min)
    :param t2: time for partical to travel through transition zone from flame tip to bouyant plume, from eq_37 (min)
    :param t3: time for partical to travel inside bouyant plume, from eq_38 (min)
    :return: t_t, time required for partical to travel upward from source (min)
    """
    return t_o + t1 + t2 + t3

def eq_36(z_F, v_o, w_F):
    """
    :param z_F: flame height (m)
    :param v_o: terminal velocity of partical (m/sec)
    :param w_F: flame gas velocity (m/sec)
    :return: t1, time for partical to travel from initial height to flame tip (min)
    """
    # z_o = .4306
    # B = 40
    ln_stuff = np.log((1-v_o/w_F)/((.4306/z_F)**.5 - (v_o)/(w_F)))
    return 1 - (.4306/z_F)**.5 + (v_o/w_F)*ln_stuff

def eq_37(Dp, z_F):
    """
    :param Dp: Partical diameter (m)
    :param z_F: flame height (m)
    :return: t2, time for partical to travel through transition zone from flame tip to bouyant plume (min)
    """
    ln_stuff = np.log(1 + 1/(1-(Dp/z_F)**.5))
    return .2 + 40*((Dp/z_F)**.5)*(1 + 40*((Dp/z_F)**.5)*ln_stuff)

def eq_38(v_o, w_F, z, Dp, a, b, z_F):
    """
    :param z: particul height (m)
    :param v_o: terminal velocity of partical (m/sec)
    :param w_F: flame gas velocity (m/sec)
    :param Dp: Partical diameter (m)
    :param z_F: flame height (m)
    :param a: defines shape of eliptical fire, from eq_15 (vertical radius) (m/min)
    :param b: defines shape of eliptical fire, from eq_16 (horizontal radius) (m/min)
    :return: t3, time for partical to travel inside bouyant plume (min)
    """
    r = ((b + z/z_F)/(a))**.5
    x = v_o/w_F
    ln_stuff = np.log((1-.8*(x))/(1-.8*r*(x)))
    return 5.963/(.8*x)*(ln_stuff - .8*x*(r-1) - .5*((.8*x)**2)*(r-1)**2)
    
def eq_39(z0, t, tau, v_o0):
    """
    :param z0: initial height of partical
    :param t: current time (min)
    :param v_o0: terminal velocity, from eq_41 (m/sec)
    :param tau: from eq_40
    :return: z_t, partical height at time t (m)
    """
    return z0 - v0*(t/tau - .5*(t/tau)**2)

def eq_40(v_o0):
    """
    :param v_o0: terminal velocity, from eq_41 (m/sec)
    :return: tau
    """
    return (4*1.2*v_o0)/(.0064*3.1415*9.8)

def eq_41(Dp):
    """
    :param Dp: Partical diameter (m)
    :return: v_o (okay, so what is v_o0?)
    """
    return ((3.1415*.3*9.8*Dp)/(2*1.2*.0012))**.5

def eq_42(U_H, z, H):
    """
    :param U_H: wind speed at height H, from eq_43 (m/sec)
    :param z: height of partical (m)
    :param H: height of forest canopy (m)
    :return: dXdt, velocity in X direction of partical (m/sec)
    """
    return U_H*np.log(z/.4306)/np.log(H/.4306)

def eq_43(U_20pH, H):
    """
    :param U_20pH: windspeed at twenty feet (6.1 m) over tree height (m/sec)
    :param H: height of canopy (m)
    :return: U_H, wind speed at height H (m/sec)
    """
    return U_20pH/np.log((20+1.18*H)/(.43*H))

# Equation Set 8
# Misc weather stuff

def eq_44(U_20pH, H, f):
    """
    :param U_20pH: windspeed at twenty feet (6.1 m) over tree height (m/sec)
    :param H: height of canopy (m)
    :param f: crown filling fraction, from eq_45
    :return: U, open wind speed (m/sec)
    """
    return .555*U_20pH/(((f*3.28*H)**.5)*np.log((20+1.18*H)/(.43*H)))

def eq_45(C):
    """
    :param C: canopy cover from spatial inputs
    :return: f, crown filling fraction
    """
    return 3.1415*C/(1200)